In [1]:
## database imports
from database.market import Market
from database.adatabase import ADatabase

## data processor import
from processor.processor import Processor as p

## risk and return class imports
from returns.products import Products as returns_products_class
from risk.weekly_risk import WeeklyRisk as risk_class
from returns.weekly_returns import WeeklyReturns as returns_class

## backtester import
from backtester.weekly_backtester import WeeklyBacktester as b

## additional imports
from datetime import datetime
import pandas as pd
from tqdm import tqdm

In [2]:
start_date = datetime(2023,1,1)
end_date = datetime.now()

In [3]:
market = Market()
speculation_db = ADatabase("spec")
classification_db = ADatabase("spec_classification")
class_spec = ADatabase("class_spec")
weeklies_db = ADatabase("current_weekly")

In [4]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [5]:
tyields = returns_products_class.tyields()
bench_returns = returns_products_class.spy_bench()

In [6]:
new_prices = []
market.connect()
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        ticker_sim = market.retrieve_ticker_prices("prices",ticker)
        ticker_sim = returns_class.returns(ticker_sim)
        completed = risk_class.risk(ticker_sim,bench_returns)
        new_prices.append(completed)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
price_returns = pd.concat(new_prices)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:49<00:00,  9.89it/s]


In [7]:
speculation_db.connect()
simulation = speculation_db.retrieve("predictions")
speculation_db.disconnect()

In [8]:
classification_db.connect()
classification_simulation = classification_db.retrieve("predictions")
classification_db.disconnect()

In [9]:
simulation = price_returns.merge(simulation.drop("adjclose",axis=1)[["year","week","ticker","prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose"],axis=1)[["year","week","ticker","classification_prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(tyields[["date","weekly_yield"]],on="date",how="left")

In [10]:
sim = simulation.dropna()

In [11]:
sim.sort_values(["date","ticker"],inplace=True)
sim = sim.groupby(["date","ticker","classification_prediction"]).mean().reset_index()

In [12]:
trades = []
values = [True,False]
classifications = [True,False]
ceilings = [True,False]
floors = [True,False]
parameters = []
positions = 10
strats = ["spec","rolling","window"]
for strat in strats:
    for value in values:
        for classification in classifications:
            for ceiling in ceilings:
                for floor in floors:
                    parameter = {
                                "value":value
                                 ,"classification":classification
                                 ,"ceiling":ceiling
                                 ,"floor":floor
                                ,"strat":strat
                                }
                    parameters.append(parameter)

In [14]:
weeklies_db.connect()
weeklies_db.drop("trades")
for parameter in tqdm(parameters):
    backtest_data = sim.copy().dropna()
    strat = parameter["strat"]
    backtest_data = returns_class.returns_backtest(strat,backtest_data)
    b.backtest(backtest_data.copy(),positions,parameter,start_date,end_date,weeklies_db)
weeklies_db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [01:13<00:00,  1.53s/it]
